# Preprocessing
This notebook has the results of the preprocessing of the data

In [111]:
#packages
import numpy as np
import pandas as pd
import datetime as dt
import geopandas
import bokeh

In [112]:
#we first need to load the data from ./data
#assignment 1 focusses on the sales, so let's import that data into a pandas dataframe

append_string_list1 = ["202106","202107","202108","202109","202110","202111","202112"]#the names we try to process
append_string_list2 = []#the leftover names
final_df_list = []
needed_columns = ['Transaction Date','Transaction Type','Product id','Sku Id','Buyer Country','Buyer Postal Code','Amount (Merchant Currency)']
needed_columns_lower = [col.lower() for col in needed_columns]#the lower case names
needed_columns_set = set(needed_columns_lower)#a set of this list, to check with subset later

for name in append_string_list1:
    if not final_df_list:#we have the first df
        first_df = pd.read_csv(f"./data/sales_{name}.csv")
        first_df.columns = first_df.columns.str.lower()#always lower case column names
        final_df_list.append(first_df[needed_columns_lower])
    else:
        #we already have the first dataframe, now we need to check for every column type if it's in the columns list
        next_df = pd.read_csv(f"./data/sales_{name}.csv")
        next_df.columns = next_df.columns.str.lower()#convert to lower case
        columns_set = set(next_df.columns)
        if needed_columns_set.issubset(columns_set):
            final_df_list.append(next_df[needed_columns_lower])
        else:
            append_string_list2.append(name)
            print(f"{name} is not a subset")

#let's do some data type processing on the list we already have
for i in range(len(final_df_list)):
    final_df_list[i].loc[:,'transaction date'] = pd.to_datetime(final_df_list[i]['transaction date']).dt.date#convert to datetime
    final_df_list[i] = final_df_list[i].loc[final_df_list[i]['transaction type'] == "Charge"] #drop columns that aren't charge
    final_df_list[i] = final_df_list[i].loc[final_df_list[i]['product id'] == "com.vansteinengroentjes.apps.ddfive"] #only use this product id


pd.concat(final_df_list).to_csv("finaldf.csv")

202111 is not a subset
202112 is not a subset


In [113]:
#Now we need to process the append_string_list2. The data in here has a different structure than our other data
#Because of this, we will construct these from the ground up, and we will zoom in on just these datasets (so without a loop)
df1 = pd.read_csv(f"./data/sales_{append_string_list2[0]}.csv")
df2 = pd.read_csv(f"./data/sales_{append_string_list2[1]}.csv")
#let's check if the column names are the same
df1.columns = df1.columns.str.lower()#convert to lower case
df2.columns = df1.columns.str.lower()#convert to lower case
columns_set1 = set(df1.columns)
columns_set2 = set(df2.columns)
print()

if columns_set1.issubset(columns_set2) or columns_set2.issubset(columns_set1):
#we can append to the list
    last_df = pd.concat([df1,df2])
else:
    print("not a subset")

In [114]:
#These last 2 have the same structure, we only need to create a dataframe that matches the first one
#We go column by column to see if it matches

#Transaction date column is called 'order charged timestamp', let's change that
last_df.rename(columns={"order charged date" : "transaction date"}, inplace=True)
#this column also needs to be changed to the datetime format
last_df.loc[:,'transaction date'] = pd.to_datetime(last_df['transaction date']).dt.date

In [115]:
# Transaction type is now called financial status:
last_df.rename(columns={"financial status" : "transaction type"}, inplace=True)

#but also the data type is different. Instead of being "Charge" and "Google Fee" we now have "Charged" and "Refund".
#Since we end up only using the "Charge" entries anyways we will change the name "Charged" to "Charge" and we leave out "Refund".
last_df = last_df[last_df["transaction type"] == "Charged"]
last_df["transaction type"] = last_df["transaction type"].replace('Charged', 'Charge', regex=False)

#product id and sku id are the same, so we only have to filter on the product id:
last_df = last_df.loc[last_df['product id'] == "com.vansteinengroentjes.apps.ddfive"] #only use this product id

In [116]:
#Country of Buyer needs to be renamed to "Buyer Country"
last_df.rename(columns={"country of buyer" : "buyer country"}, inplace=True)

In [117]:
#we first need to load the data from ./data
#assignment 1 focusses on the sales, so let's import that data into a pandas dataframe

append_string_list1 = ["202106","202107","202108","202109","202110","202111","202112"]#the names we try to process
append_string_list2 = []#the leftover names
final_df_list = []
needed_columns = ['Transaction Date','Transaction Type','Product id','Sku Id','Buyer Country','Buyer Postal Code','Amount (Merchant Currency)']
needed_columns_lower = [col.lower() for col in needed_columns]#the lower case names
needed_columns_set = set(needed_columns_lower)#a set of this list, to check with subset later

for name in append_string_list1:
    if not final_df_list:#we have the first df
        first_df = pd.read_csv(f"./data/sales_{name}.csv")
        first_df.columns = first_df.columns.str.lower()#always lower case column names
        final_df_list.append(first_df[needed_columns_lower])
    else:
        #we already have the first dataframe, now we need to check for every column type if it's in the columns list
        next_df = pd.read_csv(f"./data/sales_{name}.csv")
        next_df.columns = next_df.columns.str.lower()#convert to lower case
        columns_set = set(next_df.columns)
        if needed_columns_set.issubset(columns_set):
            final_df_list.append(next_df[needed_columns_lower])
        else:
            append_string_list2.append(name)
            print(f"{name} is not a subset")

#let's do some data type processing on the list we already have
for i in range(len(final_df_list)):
    final_df_list[i].loc[:,'transaction date'] = pd.to_datetime(final_df_list[i]['transaction date']).dt.date#convert to datetime
    final_df_list[i] = final_df_list[i].loc[final_df_list[i]['transaction type'] == "Charge"] #drop columns that aren't charge
    final_df_list[i] = final_df_list[i].loc[final_df_list[i]['product id'] == "com.vansteinengroentjes.apps.ddfive"] #only use this product id


pd.concat(final_df_list).to_csv("finaldf.csv")

202111 is not a subset
202112 is not a subset


In [118]:
#"postal code of buyer" needs to be renamed to "buyer postal code"
last_df.rename(columns={"postal code of buyer" : "buyer postal code"}, inplace=True)

In [119]:
#We need to change "charged amount" to "amount (merchant currency)". But we also need to convert the prices
#to euros. We assume in the original data that the amount (merchant currency) is in euros, per the assignment.
#so we need to convert these values to euros using the conversion of currencies to euros.
from currency_converter import CurrencyConverter
c = CurrencyConverter(fallback_on_wrong_date=True, fallback_on_missing_rate=True)
#we will this currencyconverter. However, not all currencies are supported, the ones that aren't we will have to
#remove from the dataset. We can check this with the following set:
old_length = len(last_df)
last_df = last_df[last_df['currency of sale'].isin(c.currencies)].copy()
print(f"we lost {old_length-len(last_df)} rows of data")
last_df['amount (merchant currency)'] = last_df.apply( lambda row: c.convert(row['charged amount'],row["currency of sale"], date=row["transaction date"]), axis=1)

we lost 3 rows of data


In [120]:
#now, let's filter on our used columns and append the prior dataframe to it to get the full dataframe
final_df1 = pd.concat(final_df_list,ignore_index=True) #we concatinate the entire list to one dataframe
final_df2 = last_df[needed_columns_lower]

sales_df = pd.concat([final_df1, final_df2],ignore_index=True)
sales_df.to_csv("sales_df.csv")